In [532]:
!pip install tensorflow-addons==0.11.1

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [533]:
import numpy as np 
import matplotlib.pyplot as plt

import tensorflow_addons as tfa
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

from sklearn.utils import class_weight as cw

In [534]:
tf.__version__

'2.3.0'

In [590]:
batch_size = 16
original_img_size = (256, 2600)
model_img_height = 256
model_img_width = 512
seed = 1
storage_dir = '../input/birdmel/train_img_final/train_img_final'
AUTOTUNE = tf.data.experimental.AUTOTUNE
crop_height = model_img_height
crop_width = model_img_width

In [591]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    storage_dir,
    validation_split=0.1,
    subset="training",
    seed=seed,
    image_size=original_img_size,
    batch_size=batch_size,
    label_mode='categorical'
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    storage_dir,
    validation_split=0.1,
    subset="validation",
    seed=seed,
    image_size=original_img_size,
    batch_size=batch_size,
    label_mode='categorical',
)

Found 72956 files belonging to 265 classes.
Using 65661 files for training.
Found 72956 files belonging to 265 classes.
Using 7295 files for validation.


In [592]:
def build_model(num_classes, img_width=model_img_width, img_height=model_img_height):
    inputs = layers.Input(shape=(img_height, img_width, 3))
    model = EfficientNetB4(include_top=False, input_tensor=inputs)

    x = layers.GlobalMaxPooling2D(name="max_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.5
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(num_classes, activation="softmax", name="pred")(x)

    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    model.compile(
        optimizer=Adam(learning_rate=7.5e-6), 
        loss="categorical_crossentropy", 
        metrics=[tfa.metrics.F1Score(num_classes = num_classes, average = 'micro')]
    )

    return model

In [593]:
random_crop_layer = preprocessing.RandomCrop(crop_height, crop_width)
random_translate_layer = preprocessing.RandomTranslation(height_factor=0, width_factor=0.1, fill_mode='constant')
random_contrast_layer = preprocessing.RandomContrast(factor=0.1)
rescale_layer = preprocessing.Rescaling(scale=1./255)

def resize(images, height=model_img_height, width=model_img_width):
    return tf.image.resize_with_crop_or_pad(images, target_height=height, target_width=width)

def random_mask(images, time_mask_length, freq_mask_length):
    if time_mask_length > 0:
        images = tfa.image.random_cutout(
            images,
            mask_size=(model_img_height*3, time_mask_length)
        )
    
    if freq_mask_length > 0: 
        images = tfa.image.random_cutout(
            images,
            mask_size=(freq_mask_length, model_img_width*3)
        )
    
    return images

def augment_image_train(images):    
    images = random_crop_layer(images)
    images = random_contrast_layer(images)
    images = random_mask(images, time_mask_length=30, freq_mask_length=20)
    images = random_translate_layer(images)
    images = rescale_layer(images)

    return images

def augment_image_test(images):
    images = resize(images)
    images = rescale_layer(images)
    
    return images

In [594]:
def prepare(ds, augment=False):
    # Use data augmentation only on the training set
    if augment:
        ds = ds.map(lambda x, y: (augment_image_train(x), y), num_parallel_calls=AUTOTUNE)
    else: 
        ds = ds.map(lambda x, y: (augment_image_test(x), y), num_parallel_calls=AUTOTUNE)

    # Use buffered prefecting on all datasets
    return ds.prefetch(buffer_size=AUTOTUNE)

In [595]:
class_num = len(train_ds.class_names)

In [596]:
train_ds = prepare(train_ds, augment=True)
val_ds = prepare(val_ds)

In [597]:
# image = next(iter(train_ds))[0][0, :, :, :]
# plt.imshow(image)

In [598]:
# image.shape

In [599]:
train_datagen = ImageDataGenerator(dtype=np.float16)

train_generator = train_datagen.flow_from_directory(
    directory=storage_dir,
    class_mode="categorical",
    target_size=original_img_size,
    batch_size=batch_size
)

class_weights = cw.compute_class_weight(
    'balanced',
    np.unique(train_generator.classes), 
    train_generator.classes
)

class_weights = {i : class_weights[i] for i in range(len(class_weights))}

del train_datagen
del train_generator

Found 72956 images belonging to 265 classes.


In [604]:
net = build_model(class_num)

In [605]:
net.load_weights( './efn_b4_ckpt_tf_2_3_0.h5')

In [606]:
net.save('./efn_b4_tf_2_3_0_final.h5')

In [602]:
model_check = ModelCheckpoint(
    'efn_b4_ckpt_tf_2_3_0.h5',
    monitor='val_f1_score', 
    verbose=0, 
    save_best_only=True, 
    save_weights_only=True,
    mode='max',
    period=1,
)

reduce_LR = ReduceLROnPlateau(
    monitor='val_f1_score',
    factor=0.1,
    patience=7, 
    min_lr=1e-6,
    min_delta=0.005
)

In [603]:
net.fit(
    train_ds,
    validation_data = val_ds,
    epochs = 25,
    class_weight = class_weights,
    callbacks = [reduce_LR, model_check]
)

Epoch 1/25
4104/4104 [==============================] - 2047s 499ms/step - loss: 0.8513 - f1_score: 0.7305 - val_loss: 1.2996 - val_f1_score: 0.7287
Epoch 2/25
4104/4104 [==============================] - 2042s 498ms/step - loss: 0.8525 - f1_score: 0.7297 - val_loss: 1.2752 - val_f1_score: 0.7324
Epoch 3/25
3842/4104 [===========================>..] - ETA: 2:04 - loss: 0.8664 - f1_score: 0.7230

KeyboardInterrupt: 